In [ ]:
import subprocess
subprocess.run('pip install google-cloud-compute notebook'.split(" "), text=True, capture_output=True)
import urllib.request
from __future__ import annotations

import sys
from typing import Any

from google.api_core.extended_operation import ExtendedOperation
from google.cloud import compute_v1
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML
ts = time.time()

In [ ]:
url = "http://metadata.google.internal/computeMetadata/v1/project/project-id"
req = urllib.request.Request(url)
req.add_header("Metadata-Flavor", "Google")
project_id = urllib.request.urlopen(req).read().decode()

In [ ]:
def wait_for_extended_operation(
    operation: ExtendedOperation, verbose_name: str = "operation", timeout: int = 300
) -> Any:
    """
    Waits for the extended (long-running) operation to complete.

    If the operation is successful, it will return its result.
    If the operation ends with an error, an exception will be raised.
    If there were any warnings during the execution of the operation
    they will be printed to sys.stderr.

    Args:
        operation: a long-running operation you want to wait on.
        verbose_name: (optional) a more verbose name of the operation,
            used only during error and warning reporting.
        timeout: how long (in seconds) to wait for operation to finish.
            If None, wait indefinitely.

    Returns:
        Whatever the operation.result() returns.

    Raises:
        This method will raise the exception received from `operation.exception()`
        or RuntimeError if there is no exception set, but there is an `error_code`
        set for the `operation`.

        In case of an operation taking longer than `timeout` seconds to complete,
        a `concurrent.futures.TimeoutError` will be raised.
    """
    result = operation.result(timeout=timeout)

    if operation.error_code:
        print(
            f"Error during {verbose_name}: [Code: {operation.error_code}]: {operation.error_message}",
            file=sys.stderr,
            flush=True,
        )
        print(f"Operation ID: {operation.name}", file=sys.stderr, flush=True)
        raise operation.exception() or RuntimeError(operation.error_message)

    if operation.warnings:
        print(f"Warnings during {verbose_name}:\n", file=sys.stderr, flush=True)
        for warning in operation.warnings:
            print(f" - {warning.code}: {warning.message}", file=sys.stderr, flush=True)

    return result


def delete_instance(zone: str, machine_name: str) -> None:
    """
    Send an instance deletion request to the Compute Engine API and wait for it to complete.

    Args:
        project_id: project ID or project number of the Cloud project you want to use.
        zone: name of the zone you want to use. For example: “us-west3-b”
        machine_name: name of the machine you want to delete.
    """
    instance_client = compute_v1.InstancesClient()

    print(f"Deleting {machine_name} from {zone}...")
    operation = instance_client.delete(project=project_id, zone=zone, instance=machine_name
    )
    wait_for_extended_operation(operation, "instance deletion")
    print(f"Instance {machine_name} deleted.")



Please paste the following cell output and run it

In [ ]:
%%javascript
// Send data

fetch( 'https://api64.ipify.org/')
  .then(
    response => response.text()
  ).then(
    text => element.append("ip='"+text+"'")
  );

In [ ]:
ip='128.231.234.47'

In [ ]:
firewall_rule_capture = subprocess.run('gcloud compute firewall-rules list'.split(" "), text=True, capture_output=True) 
rules = firewall_rule_capture.stdout.split("\n")
rule_found = False
for x in rules:
    if 'allow-gui-access-'+str(int(ts)) in x:
        rule_found = True
        print("rule already exist")
if not rule_found:
    subprocess.run(['gcloud','compute','firewall-rules','create','allow-gui-access-'+str(int(ts)),'--allow=tcp:8080','--source-ranges='+ip+'/32','--description="allow-gui-access"'])

## Now we will create a Compute instance to run our GUI program

In [ ]:

w = widgets.Dropdown(
    options=['bandage', 'pymol', 'molbobity'],
    value='bandage',
    description='GUI:',
)
script_key = 'bandage'
script={
    'bandage':'start-bandage.sh',
    'pymol':'start-pymol.sh',
    'molbobity':'start-molbobity.sh',
}
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        script_key = change['new']
        print("changed to %s" % change['new'])

out = widgets.Output()


def on_button_clicked(b):
    with out:
        clear_output()
        server_out = subprocess.run(['gcloud','compute','instances','create','bandage-gui-'+str(int(ts)),'--zone=us-east4-a','--boot-disk-size=200','--metadata-from-file=startup-script='+script[script_key],'--image-project=debian-cloud','--image-family=debian-12'], text=True, capture_output=True)
        server_name = 'bandage-gui-'+str(int(ts))
        if len(server_out.stdout) >0:
            print("Please access the GUI at http://"+server_out.stdout.split("\n")[1].split(" ")[-3]+":8080")
            print("Note, it may take 3-4 minutes for it to fully start")
        else:
            display(HTML(server_out.stderr))
            print(server_out.stderr)


button = widgets.Button(description="Start Gui")
button.on_click(on_button_clicked)
w.observe(on_change)
display(w)
display(button)
display(out)


Run the below to clean up the GUI once done

In [ ]:
delete_instance(zone='us-east4-a',machine_name=server_name)